In [28]:
import ase.io as aio

import os
import numpy as np
import qml
import ase
import sys
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/hitp/')
sys.path.insert(0, '/home/sahre/git_repositories/APDFT/prototyping/atomic_energies/')
from prepare_calculations import *
# import importlib
# importlib.reload()
import explore_qml_data as eqd
import utils_qm as uqm
import glob
import cpmd_io

In [ ]:
cpmd_io.fix_input_file()

In [2]:
def create_run_dir(path, atom_symbols, nuc_charges, positions, lambda_value, valence_charges = None, integer_electrons = False):

    # create directory if not exists
    os.makedirs(path, exist_ok=True)

    # set parameters independent of lambda value
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(positions, np.array([boxsize, boxsize, boxsize])/2)

    # lambda dependent quantities

    # add correct number of electrons such that system stays isoelectronic to target molecule\
    charge = calculate_charge(lambda_value, num_ve, valence_charges, integer_electrons)

    # generate input file
    input_path = os.path.join(path, 'run.inp')
    
    # start from random wavefunction as initial guess if lambda < 0.5 otherwise the calculation will crash
    if type(lambda_value) == float:
        lam_min = lambda_value
    else:
        lam_min = np.amin(lambda_value)
        
    if lam_min > 0.501:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
    else:
        write_input(atom_symbols['elIdx'], charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

    # generate pp-files
    if type(lambda_value) == float:
        write_pp_files_compound(atom_symbols, lambda_value, path, pp_dir, pp_type)
    else:
        write_pp_files_compound_partial(atom_symbols, lambda_value, path, pp_dir, pp_type)
        
def parse_xyz_for_CPMD_input(path2xyz):
    # get structure information from xyz file
    molecule = aio.read(path2xyz)

    atom_symbolsEl = []
    atom_symbolsIdx = []
    for i, el in enumerate(molecule.get_chemical_symbols()):
        atom_symbolsEl.append(el)
        atom_symbolsIdx.append(el + str(i+1))
    atom_symbols = {'el':atom_symbolsEl, 'elIdx':atom_symbolsIdx}
    nuc_charges = molecule.get_atomic_numbers()
    valence_charges = eqd.get_val_charges(nuc_charges)
    positions = molecule.get_positions()
    return(atom_symbols, nuc_charges, positions, valence_charges)

In [33]:
for ve, base_lambda in zip(['ve_02', 've_03', 've_05', 've_06'], np.array([2,3,5,6])/8):

    xyz_path = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/GEOMETRY.xyz'
    atom_symbols, nuc_charges, positions, valence_charges = parse_xyz_for_CPMD_input(xyz_path)
    
    dlambda = 1e-5 # finite difference
    increments = np.array([[dlambda, dlambda, dlambda], [dlambda, 0, 0], [0,dlambda,0], [0,0,dlambda]])
    
    for derivative, lambdas in zip(['lam', 'H1', 'H2', 'O3'], increments):
        for sign, label in zip([-1, 1], ['bw', 'fw']):
            path = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}' # path to directory where run.inp and pseudopotentials will be written to
            lambda_value = sign*lambdas + base_lambda# lambda value can be a single value (for full derivative) or list with len == number of atoms for partial derivatives
            # print(lambda_value)
    
            atom_symbols = atom_symbols # string element + integer index
            nuc_charges = nuc_charges# list of nuclear charges
            valence_charges = valence_charges
            positions = positions# atom positions


            integer_electrons = False# rescales lambda value such that number of electrons from nuclei is an integer only kept for backwards compability

            pp_dir = '/data/sahre/PP_LIBRARY'
            pp_type = 'GH_PBE'
            template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe.inp'
            template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_pbe_small_lambda.inp'

            create_run_dir(path, atom_symbols, nuc_charges, positions, lambda_value, valence_charges, integer_electrons)
            
            # copy RESTART.1 and LATEST from base directory
            restart_path_source = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/RESTART.1'
            restart_path_dest = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}/RESTART.1'
            shutil.copy(restart_path_source, restart_path_dest)
            
            latest_path_source = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/LATEST'
            latest_path_dest = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}/LATEST'
            shutil.copy(latest_path_source, latest_path_dest)
            
            # add restart option
            input_dir = f'/data/sahre/projects/atomic-energies/amons/amon_000004/{ve}/dE_d{derivative}_{label}'
            cpmd_io.fix_input_file(input_dir)

Enabled restart (from rESTART) in /data/sahre/projects/atomic-energies/amons/amon_000004/ve_02/dE_dlam_bw because there is an intact restart file. Isn't that great, sunshine?
Enabled restart (from rESTART) in /data/sahre/projects/atomic-energies/amons/amon_000004/ve_02/dE_dlam_fw because there is an intact restart file. Isn't that great, sunshine?
Enabled restart (from rESTART) in /data/sahre/projects/atomic-energies/amons/amon_000004/ve_02/dE_dH1_bw because there is an intact restart file. Isn't that great, sunshine?
Enabled restart (from rESTART) in /data/sahre/projects/atomic-energies/amons/amon_000004/ve_02/dE_dH1_fw because there is an intact restart file. Isn't that great, sunshine?
Enabled restart (from rESTART) in /data/sahre/projects/atomic-energies/amons/amon_000004/ve_02/dE_dH2_bw because there is an intact restart file. Isn't that great, sunshine?
Enabled restart (from rESTART) in /data/sahre/projects/atomic-energies/amons/amon_000004/ve_02/dE_dH2_fw because there is an int

In [27]:
import shutil
shutil.collections

In [26]:
(2/8+1e-5)*-16.667214800000000


-4.1669703721480005

In [6]:
-17.21915193--17.21914198

-9.949999999037118e-06

In [13]:
e_ref = -17.21914198

In [15]:
eo = -17.21954453
eh1 = -17.21915193
eh2 = -17.21915193
elam = -17.21956442

(eo-e_ref)/dlambda+(eh1-e_ref)/dlambda+(eh2-e_ref)/dlambda

-42.244999999851984

In [17]:
(elam - e_ref)/dlambda - ((eo-e_ref)/dlambda+(eh1-e_ref)/dlambda+(eh2-e_ref)/dlambda)

0.0009999997274690031

In [ ]:
    # get correct lambda value
    for lam_val in lambda_values:
        #print(lam_val)
        elec_pp = np.round(lam_val*num_ve) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons

        lambda_path = os.path.join(compound_path, f'lam_{lam_val}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        write_pp_files_compound_partial(atom_symbols, [lam_val1, lam_val2], lambda_path, pp_dir, pp_type)
        
        print(lambda_path)

In [ ]:
def wrapper_ase2(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve/2) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(atoms.get_positions(), np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val in lambda_values:
        #print(lam_val)
        elec_pp = np.round(lam_val*num_ve) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons

        lambda_path = os.path.join(compound_path, f'lam_{lam_val}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        
        print(lambda_path)
        
def wrapper_ase3(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda):
    """
    generates all necessary files for a cpmd calculation using an atoms object from ase as input
    """
    # calculation parameters (independent of lambda value)
    atom_symbols = ['H1', 'H2']#atoms.get_chemical_symbols()
    nuc_charges = atoms.get_atomic_numbers()
    num_ve = eqd.get_num_val_elec(nuc_charges) # get number of ve
    boxsize = get_boxsize(num_ve) # get boxsize
    num_gpts_lower, num_gpts_higher = get_gpts(num_ve/2) # get gridpoints
    num_gpts = num_gpts_higher
    # shift molecule to center of box
    coords_final = eqd.shift2center(atoms.get_positions(), np.array([boxsize, boxsize, boxsize])/2)

    # get correct lambda value
    for lam_val1, lam_val2 in lambda_values:
        #print(lam_val)
        elec_pp = int(np.round(lam_val1 + lam_val2)) # electrons coming from pseudopotentials
        charge = elec_pp-num_ve # electrons added to conserve total number of electrons
        lambda_path = os.path.join(compound_path, f'H2_lam_{lam_val2}/')
        os.makedirs(lambda_path, exist_ok=True)

        # generate input file
        input_path = os.path.join(lambda_path, 'run.inp')
        if lam_val1 > 0.501:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp, debug = False)
        else:
            write_input(atom_symbols, charge, coords_final, num_gpts, boxsize, input_path, pp_type, template_inp_small_lambda, debug = False)

        # generate pp-files
        #write_pp_files_compound(atom_symbols, lam_val, lambda_path, pp_dir, pp_type)
        write_pp_files_compound_partial(atom_symbols, [lam_val1, lam_val2], lambda_path, pp_dir, pp_type)
        print(lambda_path)

In [ ]:
# define parameters
pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'SG_LDA'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_cutoff.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_small_lambda_cutoff.inp'
compound_path = '/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched'
dlam = 0.00001
#lambda_values = np.array([0.11, 0.31, 0.41, 0.51, 0.71, 0.91]) + dlam

lambda_values = np.sort(np.concatenate((np.arange(0.0, 1.05, 0.05), np.arange(0.0, 1.05, 0.05)+dlam)))

atoms = aio.read('/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched.xyz')

wrapper_ase2(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda)

In [ ]:
# define parameters
pp_dir = '/data/sahre/PP_LIBRARY'
pp_type = 'SG_LDA'
template_inp = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_cutoff.inp'
template_inp_small_lambda = '/data/sahre/projects/atomic-energies/cpmd_params_template_lda_small_lambda_cutoff.inp'
compound_path = '/data/sahre/projects/atomic-energies/derivatives/H2/partial_derivatives_stretched/'
dlam = 0.00001

lam = np.arange(0.0, 1.05, 0.05)

lambda_values = np.array([lam, lam+dlam]).T

atoms = aio.read('/data/sahre/projects/atomic-energies/derivatives/H2/H2_stretched.xyz')

wrapper_ase3(atoms, compound_path, lambda_values, pp_dir, pp_type, template_inp, template_inp_small_lambda)

In [ ]:
np.arange(0.0, 1.05, 0.05)